### IAA - Agentes Conversacionais
Priscilla Bomfim (@priscillapreks)
30.08.2021


In [1]:
!pip install nltk
!pip install selenium
!pip install webdriver-manager
!pip install lxml

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import time
import random
import string
import bs4 as bs
import urllib.request
import re

import nltk
nltk.download('punkt')
nltk.download('rslp')# Stemming em pt-br
from nltk.corpus import stopwords
nltk.download('stopwords')# Lista de stopwords

from selenium import webdriver #Automatizador de acesso ao navegador
from webdriver_manager.chrome import ChromeDriverManager #Driver específico do Chrome
from selenium.webdriver.common.keys import Keys #Quando precisamos simular alguma tecla especial

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\preks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\preks\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\preks\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Navegar até WhatsApp Web
driver = webdriver.Chrome(ChromeDriverManager().install())
print("Acessando o WhatsApp Web...")
driver.get('https://web.whatsapp.com/')
#driver.maximize_window()
print("Escaneie o QR Code, e então pressione ENTER")
input()



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\preks\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


Acessando o WhatsApp Web...
Escaneie o QR Code, e então pressione ENTER



''

In [4]:
# Cria o corpus com web-scrapping da wikipedia
wikipedia_url = 'https://pt.wikipedia.org/wiki/Pandemia_de_COVID-19'

codigo_html = urllib.request.urlopen(wikipedia_url)
codigo_html = codigo_html.read()

# Processa o código HTML lido
html_processado = bs.BeautifulSoup(codigo_html, 'lxml')

# Busca todos parágrafos do texto
paragrafos = html_processado.find_all('p')

texto = ''

# Percorre parágrafos e concatena textos
for p in paragrafos:
  texto += p.text

# Pré-processamento do corpus
#texto = texto.lower()
texto = re.sub(r'\[[0-9]*\]', ' ', texto)
texto = re.sub(r'\s+', ' ', texto)

sentencas = nltk.sent_tokenize(texto, language='portuguese')
palavras = nltk.word_tokenize(texto, language='portuguese')

sentencas[10:15]

['Várias vacinas estão sendo desenvolvidas e distribuídas ao redor do mundo.',
 'Os tratamentos atuais se concentram nos sintomas enquanto drogas terapêuticas que inibem o vírus são desenvolvidas.',
 'Autoridades em todo o mundo responderam implementando restrições a viagens, lockdowns, controles de locais de trabalho e fechamentos de instalações.',
 'Muitos lugares também trabalharam para aumentar a capacidade de testar e rastrear os contatos dos infectados.',
 'A pandemia resultou em instabilidade social e econômica global significativa, incluindo a maior recessão global desde a Grande Depressão.']

In [5]:
# define as saudações 
saudacoes = ("oi", "olá", "bom dia", "boa tarde", "boa noite")
saudacoes_respostas = ["Olá! O que você gostaria de saber sobre a pandemia de COVID-19?",
                       "Oi! Eu sei tudo sobre a COVID-19. Qual a sua dúvida?",
                       "Olá, me faça uma pergunta sobre a pandemia de coronavírus!"]

def geradorsaudacoes(saudacao):
  for token in saudacao.split():
    if token.lower() in saudacoes:
      return random.choice(saudacoes_respostas)  

In [6]:
# Função de Stemming
def stemming(tokens):
  stemmer = nltk.stem.RSLPStemmer()
  novotexto = []
  for token in tokens:
    novotexto.append(stemmer.stem(token.lower()))
  return novotexto

# Função que remove pontuação
removePontuacao = dict((ord(punctuation), None) for punctuation in string.punctuation)

def preprocessa(mensagem):
  return stemming(nltk.word_tokenize(mensagem.lower().translate(removePontuacao), language='portuguese'))

In [7]:
# Função para lidar com consultas do usuário
def geradorrespostas(msg_usuario):
  resposta = ''
  sentencas.append(msg_usuario)

  word_vectorizer = TfidfVectorizer(tokenizer=preprocessa, stop_words=stopwords.words('portuguese'))
  all_word_vectors = word_vectorizer.fit_transform(sentencas)
  similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
  similar_sentence_number = similar_vector_values.argsort()[0][-2]

  matched_vector = similar_vector_values.flatten()
  matched_vector.sort()
  vector_matched = matched_vector[-2]

  if vector_matched == 0:
    resposta = resposta + "Me desculpe, não entendi. Poderia reformular sua pergunta?"
    return resposta
  else:
    resposta = resposta + sentencas[similar_sentence_number]
    return resposta

In [8]:
# Função para gerar as respostas ao usuário

def interacao_whatsapp(mensagem):

  continue_dialogue = True

  while (continue_dialogue == True):
    # Obtém entrada do usuário
    human_text = mensagem.lower()

    if human_text != 'tchau':
      if human_text == 'obrigado' or human_text == 'obrigada' or human_text == 'valeu':
        continue_dialogue = False
        print("[BOT]: Disponha. Quando tiver dúvidas, é só chamar!")
      else:
        if geradorsaudacoes(human_text) != None:
          print(geradorsaudacoes(human_text))
          continue_dialogue = False
        else:
          print("[BOT]: ", end="")
          print(geradorrespostas(human_text))
          sentencas.remove(human_text)
          continue_dialogue = False
    else:
      continue_dialogue = False
      print("[BOT]: Tchau. Até a próxima.")

In [9]:
# Integração com WhatsApp web

while True:  
    print("Buscando novas mensagens...")
    time.sleep(10)
    
    try:
        # Todos contatos que estão sinalizados com novas mensagens tem o atributo aria-label
        # Aqui buscamos todos eles
        contatos = driver.find_elements_by_css_selector("span[aria-label]")
        
        # Percorre contatos com novas mensagens
        for contato in contatos:
            
            # Clica no contato para que possamos acessar o campo de mensagem
            contato.click()        
            
            todasMensagens = driver.find_elements_by_css_selector(".copyable-text")            
            ultimaMensagem = todasMensagens[-2]     #[ len(todasMensagens) - 2 ]
            print('Mensagem do usuário: ', ultimaMensagem.text)
            
            
            ## RESPOSTA ##
             
            mensagem = ultimaMensagem.text
            resposta = interacao_whatsapp(mensagem)
                         
            campoMensagem = driver.find_elements_by_css_selector("div[contenteditable='true']")[1]
                        
            campoMensagem.click()
            time.sleep(2)
            campoMensagem.send_keys(resposta) # texto resposta para o bot
            time.sleep(5)
            campoMensagem.send_keys(Keys.ENTER)

    except:
        print("Sem mensagens novas...")
        # Coloca um timer para continuar depois de X segundos
        time.sleep(5)

Buscando novas mensagens...
Buscando novas mensagens...
Buscando novas mensagens...
Buscando novas mensagens...
Buscando novas mensagens...
Mensagem do usuário:  Qual a origem da pandemia do coronavírus?
[BOT]: É provável que o vírus tenha tido origem numa mutação dos coronavírus de morcegos.
Sem mensagens novas...
Buscando novas mensagens...
Mensagem do usuário:  Oi
Olá, me faça uma pergunta sobre a pandemia de coronavírus!
Sem mensagens novas...
Buscando novas mensagens...
Sem mensagens novas...
Buscando novas mensagens...


KeyboardInterrupt: 


---    


#### Interagindo com o chatbot
Faça perguntas como:
*  *Qual a origem da pandemia do coronavírus?*
*  *Quais os tratamentos para Covid-19?*
*  *Quais são as medidas preventivas recomendadas?*
*  *Perda de olfato pode ser COVID-19?* 
*  *Quais vacinas existem contra COVID-19?* 

  
    
    
---

In [10]:
interacao_whatsapp('Oi')

Olá! O que você gostaria de saber sobre a pandemia de COVID-19?


In [11]:
interacao_whatsapp('Quais os tratamentos para Covid-19?')

[BOT]: Não existem medicamentos antivirais aprovados para o tratamento de COVID-19, embora estejam vários a ser desenvolvidos e a serem testados medicamentos já existentes.


In [12]:
interacao_whatsapp('Qual a origem da pandemia do coronavírus?')

[BOT]: É provável que o vírus tenha tido origem numa mutação dos coronavírus de morcegos.


In [13]:
interacao_whatsapp('Perda de olfato pode ser COVID-19?')

[BOT]: A perda de olfato e paladar são também sintomas comuns da COVID-19.


In [14]:
interacao_whatsapp('Quais vacinas existem contra COVID-19?') 

[BOT]: Em março de 2021, 12 vacinas foram autorizadas por pelo menos uma autoridade reguladora nacional para uso público: duas vacinas de RNA (a vacina da Pfizer–BioNTech e a vacina da Moderna), quatro vacinas inativadas convencionais (BBIBP-CorV, CoronaVac, Covaxin e CoviVac), quatro vacinas de vetor viral (Sputnik V, a vacina Oxford–AstraZeneca, a Convidecia e a vacina Johnson & Johnson) e duas vacinas de subunidade proteica (EpiVacCorona e RBD-Dimer).


In [15]:
interacao_whatsapp('Obrigada') 

[BOT]: Disponha. Quando tiver dúvidas, é só chamar!


In [16]:
interacao_whatsapp('Tchau') 

[BOT]: Tchau. Até a próxima.
